# Filtering a Preexisting Trajectory or Data Series

In some cases, there is no need to perform control based on the particle belief and instead the goal is to filter a previously-generated sequence of measurements to remove noise or reconstruct hidden state variables. This tutorial will illustrate use of the `runfilter` function for that purpose.

In [13]:
using Distributions
using Random
using DelimitedFiles
using ParticleFilters
using Plots
plotly()

Plots.PlotlyBackend()

## Problem

We will use the 

In [14]:
const dt = 0.2
const mu = 0.8
const sigma = 1.0

function f(x, u, rng)
    xdot = [x[2], mu*(1-x[1]^2)*x[2] - x[1] + u + randn(rng)]
    return x + dt*xdot
end

g(x1, u, x2, y) = pdf(Normal(sigma), y-x2[1])

g (generic function with 1 method)

In [15]:
rng = MersenneTwister(1)

k = 100
h(x, rng) = x[1] + sigma*randn(rng)

x = [3.0, 0.0]
xs = Array{typeof(x)}(undef, k)
us = zeros(k)
ys = zeros(k)
for t in 1:k
    x = f(x, us[t], rng)
    ys[t] = h(x, rng)
    xs[t] = x
end

using DelimitedFiles
writedlm("x.txt", xs)
writedlm("y.txt", ys)
writedlm("u.txt", us)

In [16]:
ys = vec(readdlm("y.txt"))
us = vec(readdlm("u.txt"))

n = 1000
m = ParticleFilterModel{eltype(xs)}(f, g)
fil = SIRParticleFilter(m, n)

b0 = ParticleCollection([[10.0*(rand()-0.5), 0.0] for i in 1:n])
bs = runfilter(fil, b0, us, ys);

In [17]:
xmat = readdlm("x.txt")
xs = [xmat[i,:] for i in 1:size(xmat, 1)]

plot([first(xk) for xk in xs], label="x")
plot!(ys, label="y")
plot!([first(m) for m in map(mean, bs)], label="mean(b)")